In [3]:
# load required libraries
library(tidyverse)
library(janitor)
library(dplyr)
library(ggplot2)
library(skimr)
library(purrr)
library(lubridate)

source("../../R/apply_factors.R")
source("../../R/analysis_helpers.R")
source("../../R/temporal_helpers.R")
tables <- list(
  Orders   = readr::read_csv("../../data/processed/Orders.csv"),
  Returns  = readr::read_csv("../../data/processed/Returns.csv"),
  People   = readr::read_csv("../../data/processed/People.csv")
)
tables <- apply_factors(tables)
orders <- tables$Orders 
returns <- tables$Returns
people <- tables$People

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.6
✔ forcats   1.0.1     ✔ stringr   1.6.0
✔ ggplot2   4.0.1     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.2.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘janitor’


The following objects are masked from ‘package:stats’:

    chisq.test, fisher.test


Rows: 51290 Columns: 21
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (13): order_id, ship_mode, customer_name, segment, state, country, mark...
dbl   (6): sales, quantity, discount, profit, shipping_cost, year
date  (2): order_date, ship_date

ℹ Use `spec()` to retrieve the full column specification f

# Regional Accountability Overview

### Q1. How does sales performance vary across regions managed by different individuals?

In [4]:
orders_with_managers <- orders %>%
    left_join(people, by = "region")

manager_sales_performance <- orders_with_managers %>%
    group_by(person, region) %>%
    summarise(
        total_sales = sum(sales),
        total_profit = sum(profit),
        n_orders = n(),
        .groups = "drop"
    ) %>%
    arrange(desc(total_sales))

manager_sales_performance

person,region,total_sales,total_profit,n_orders
<chr>,<fct>,<dbl>,<dbl>,<int>
Anna Andreadi,Central,2822302.52,311403.98,11117
Chuck Magee,South,1600907.04,140355.77,6645
Jack Lebron,North,1248165.60,194597.95,4785
Anthony Jacobs,Oceania,1100184.61,121666.64,3487
Alejandro Ballentine,Southeast Asia,884423.17,17852.33,3129
Shirley Daniels,North Asia,848309.78,165578.42,2338
NA,EMEA,806161.31,43897.97,5029
Deborah Brumfield,Africa,783773.21,88871.63,4587
Nora Preis,Central Asia,752826.57,132480.19,2048


### Q2. Are differences in regional performance driven by order volume or order value?

In [ ]:
manager_volume_value <- orders_with_managers %>%
    group_by(person, region) %>%
    summarise(
        total_sales = sum(sales),
        n_orders = n(),
        avg_order_value = total_sales / n_orders,
        .groups = "drop"
    ) %>%
    arrange(desc(total_sales))

manager_volume_value

person,region,total_sales,n_orders,avg_order_value
<chr>,<fct>,<dbl>,<int>,<dbl>
Anna Andreadi,Central,2822302.52,11117,253.8727
Chuck Magee,South,1600907.04,6645,240.9190
Jack Lebron,North,1248165.60,4785,260.8497
Anthony Jacobs,Oceania,1100184.61,3487,315.5104
Alejandro Ballentine,Southeast Asia,884423.17,3129,282.6536
Shirley Daniels,North Asia,848309.78,2338,362.8357
NA,EMEA,806161.31,5029,160.3025
Deborah Brumfield,Africa,783773.21,4587,170.8684
Nora Preis,Central Asia,752826.57,2048,367.5911


# Performance Consistency & Risk

### Q3. Do regions managed by different individuals show similar stability over time?

In [ ]:
manager_sales_over_time <- orders_with_managers %>%
    add_time_period("order_date", period = "year") %>%
    group_by(person, region, period) %>%
    summarise(
        total_sales = sum(sales),
        .groups = "drop"
    )

manager_stability <- manager_sales_over_time %>%
    group_by(person, region) %>%
    summarise(
        mean_sales = mean(total_sales),
        sd_sales = sd(total_sales),
        cv_sales = sd_sales / mean_sales,
        .groups = "drop"
    ) %>%
    arrange(desc(cv_sales))

manager_stability


person,region,mean_sales,sd_sales,cv_sales
<chr>,<fct>,<dbl>,<dbl>,<dbl>
Deborah Brumfield,Africa,195943.30,73162.611,0.3733866
Nicole Hansen,Canada,16732.04,6198.215,0.3704399
NA,EMEA,201540.33,72417.679,0.3593210
Alejandro Ballentine,Southeast Asia,221105.79,74567.090,0.3372462
Anthony Jacobs,Oceania,275046.15,83062.357,0.3019943
Nora Preis,Central Asia,188206.64,56321.395,0.2992530
Giulietta Dortch,Caribbean,81070.22,24036.372,0.2964883
Chuck Magee,South,400226.76,117787.467,0.2943018
Jack Lebron,North,312041.40,88914.168,0.2849435


### Q4. Are underperforming regions associated with specific managers or structural factors?

In [ ]:
region_sales <- orders_with_managers %>%
    group_by(region, person) %>%
    summarise(
        total_sales = sum(sales),
        .groups = "drop"
    )

sales_threshold <- quantile(region_sales$total_sales, 0.25)

underperforming_regions <- region_sales %>%
    filter(total_sales <= sales_threshold)

underperforming_regions

region,person,total_sales
<fct>,<chr>,<dbl>
Canada,Nicole Hansen,66928.17
Caribbean,Giulietta Dortch,324280.86
East,Kelly Williams,678781.24
West,Matt Collister,725457.82


In [8]:
underperforming_diagnostics <- orders_with_managers %>%
    filter(region %in% underperforming_regions$region) %>%
    group_by(region, person) %>%
    summarise(
        avg_discount = mean(discount, na.rm = TRUE),
        avg_shipping_cost = mean(shipping_cost, na.rm = TRUE),
        avg_profit_margin = sum(profit, na.rm = TRUE) / sum(sales, na.rm = TRUE),
        .groups = "drop"
    )

underperforming_diagnostics

region,person,avg_discount,avg_shipping_cost,avg_profit_margin
<fct>,<chr>,<dbl>,<dbl>,<dbl>
Canada,Nicole Hansen,0.0000000,19.28549,0.2662166
Caribbean,Giulietta Dortch,0.1357515,21.05880,0.1066092
East,Kelly Williams,0.1453652,25.54200,0.1348340
West,Matt Collister,0.1093350,23.88769,0.1494483


# Product & Segment Exposure

### Q5. Do managers oversee regions with significantly different product or segment mixes?

In [9]:
manager_product_mix <- orders_with_managers %>%
    group_by(person, category) %>%
    summarise(
        total_sales = sum(sales, na.rm = TRUE),
        .groups = "drop"
    ) %>%
    group_by(person) %>%
    mutate(
        category_share = total_sales / sum(total_sales)
    )

manager_product_mix

person,category,total_sales,category_share
<chr>,<fct>,<dbl>,<dbl>
Alejandro Ballentine,Furniture,313386.70,0.3543402
Alejandro Ballentine,Office Supplies,241285.08,0.2728163
Alejandro Ballentine,Technology,329751.38,0.3728434
Anna Andreadi,Furniture,860417.58,0.3048637
Anna Andreadi,Office Supplies,923435.28,0.3271922
Anna Andreadi,Technology,1038449.66,0.3679441
Anthony Jacobs,Furniture,410468.00,0.3730901
Anthony Jacobs,Office Supplies,281713.63,0.2560603
Anthony Jacobs,Technology,408002.98,0.3708496


In [10]:
manager_segment_mix <- orders_with_managers %>%
    group_by(person, segment) %>%
    summarise(
        total_sales = sum(sales, na.rm = TRUE),
        .groups = "drop"
    ) %>%
    group_by(person) %>%
    mutate(
        segment_share = total_sales / sum(total_sales)
    )

manager_segment_mix

person,segment,total_sales,segment_share
<chr>,<fct>,<dbl>,<dbl>
Alejandro Ballentine,Consumer,460752.82,0.5209642
Alejandro Ballentine,Corporate,254351.71,0.2875905
Alejandro Ballentine,Home Office,169318.65,0.1914453
Anna Andreadi,Consumer,1479981.17,0.5243879
Anna Andreadi,Corporate,850363.90,0.3013015
Anna Andreadi,Home Office,491957.45,0.1743107
Anthony Jacobs,Consumer,579550.47,0.5267757
Anthony Jacobs,Corporate,322826.91,0.2934298
Anthony Jacobs,Home Office,197807.23,0.1797946
